# Reglas de Asociación
---

## Contenidos
---

+ Introducción
    - Medidas para evaluar una regla
+ Algoritmos de Aprendizaje de Reglas de Clasificación
    - Algoritmo de Recubrimiento Secuencial - PRIMS
+ Algoritmos de Aprendizaje de Reglas de Asociación
    - Algoritmo A Priori


## 1 - Introducción
---

La representación del conociemiento mediante reglas de clasificación es una alternativa a los árboles de decisión, pudiendo mapearse las unas en los otros y viceversa.  

Las reglas de asociación, a diferencias de las de clasificación que predicen la clase, **predicen valores de atributos, combinaciones de valores de atributos, o la propia clase**.  

El interés en las reglas de asociación es **descubir combinaciones de pares atributo-valor** que ocurren con **frecuencia** en un conjunto de datos.  

#### Estructura de una regla
Repasando como se describía una regla, presentan la forma:  
```
SI <antecende>
     ENTONCES <consecuente>
```  
Además, las reglas podían concatenarse a través con conjuciones lógicas como ```AND``` y ```OR```.  

Parece envidente entonces la necesidad de una **medida de la probabilidad** de que dado un antecendente, se cumpla el consecuente.  

### Medidas para evaluar una Regla
---

Hay dos medidas principales:  

#### CONFIANZA
Es la probabilidad condicional de que, dado un evento A se produzca un evento B. 
```
Confianza(A → B) = P(A|B)
```
La confianza puede entenderse entonces como el **porcentaje de ejemplo que satisfacen el antecedente y consecuente** de la regla entre aquellos que **solo satisfacaen el antecendente**.  

#### SOPORTE
Es el cociente del **número de ejemplos que cumplen el antecedente y consecuente** de la regla, entre el **número total** de ejemplos.
```
Soporte(A → B) = P(A ∪ B)
```
El soporte es interesante para detectar reglas con alta confianza pero que son poco frecuentes.    

#### COBERTURA
Es el cociente entre el **número de ejemplos que cumplen el antecedente** entre el **número total** de ejemplos.  

Utilizando un conjunto de datos reales se entenderán mejor ambas medidas

In [1]:
import numpy as np
import pandas as pd

datos = pd.read_csv('./problema_tiempo.csv', sep=';')

In [2]:
datos

,Id,Ambiente,Temperatura,Humedad,Viento,Jugar
0,E1,soleado,alta,alta,falso,no
1,E2,soleado,alta,alta,verdadero,no
2,E3,nublado,alta,alta,falso,si
3,E4,lluvioso,media,alta,falso,si
4,E5,lluvioso,baja,normal,falso,si
5,E6,lluvioso,baja,normal,verdadero,no
6,E7,nublado,baja,normal,verdadero,si
7,E8,soleado,media,alta,falso,no
8,E9,soleado,baja,normal,falso,si
9,E10,lluvioso,media,normal,falso,si


Por ejemplo, al ser tan sencillo este conjunto de datos, encontramos rápido una regla de asociación que se cumple para todas las instancias:
```
SI Ambiente = nublado       SI Temperatura = baja           
   ENTONCES jugar = si         ENTONCES humedad = normal     
```
Por tanto, la confianza de estas dos reglas será ```4/4 = 1```.  
Sus soportes, en cambio, serán de ```4/14 = 0.29```   

Hay otras reglas que no se cumplen en todos los casos, como por ejemplo:
```
SI Temperatura = media          
   ENTONCES huedad = alta 
```
La confianza de esta regla será de ```4/6 = 0.67``` y el soporte será de ```4/14 = 0.29``` al igual que para las dos anteriores reglas que tenían confianza 1!  







Cuando en casos más reales el conjunto de datos es mucho más grande, el número de combinaciones son muy elevados. Por eso, se establecen **valores mínimos de confianza y soporte** para considerar cuáles de las reglas aprendidas son relevantes.  

Dicho esto, el aprendizaje de las reglas de asociación tendrá 2 fases.  
```
(1) Encontrar aquellas reglas con una frecuencia superior al soporte mínimo
(2) De estas, seleccionar aquellas cuya confianza es superior a la confianza mínima
```

## 2 - Algoritmos de Aprendizaje de Reglas de Clasificación
---

Hemos visto que una de las formas es mapear directamente desde un árbol de decisión. Sin embargo, también existen algoritmos para el aprendizaje de reglas directamente. Entre ellos, veremos SAKFHAFA.   

### Algoritmos de Recubrimiento Secuencial
<img src='./Imagenes/alg_secuencial.png'>

De esta manera, se aprende 1 regla por iteración hasta alcanzar el conjunto de reglas.

El algoritmo **PRISM** es uno de los algoritmos de recubrimiento secuencial más simples.  


## 3 - Algoritmos de Apredinzaje de Reglas de Asociación
---

### Algoritmo Apriori
---
El algoritmo apriori pretende genererar pares **ítem-sets** (conjuntos de pares atributo-valor) que cumplan una **cobertura mínima** del número de instancias que cumplen los valores en los ítem-sets.  

Etapas:
```
(1) Generar ítem-sets
(2) Generar reglas a partir de dichos ítem-sets
```

Tenemos que genera la **clase** ```Item_Set``` para poder generar **objetos** ```item_set``` que almacenaremos en la **lista** ```item_sets[]```. 

In [3]:
import numpy as np
import pandas as pd
from itertools import combinations
from beautifultable import BeautifulTable as btable

datos = pd.read_csv('./problema_tiempo.csv', sep=';')

# Parámetros
COBERTURA_MINIMA = 3
CONFIANZA_MINIMA = 0.6
EJEMPLOS = len(datos)


# Crear Item-Sets
# ===============
class Item_Set:

    def __init__(self, clase, atributo):
        
        self.nombre = str(clase) + ' = ' + str(atributo)
        self.clases = clase
        self.atributos = atributo
        self.cobertura = 0
 
    def calcular_cobertura(self, datos):
        # Functión para calcular la cobertura frente a un conjunto de datos
        self.cobertura = int(np.sum((datos[self.clases].values == self.atributos).all(axis=1)))    
        return self.cobertura


Ahora, generamos todas las combinaciones posibles. Si Temperatura = Alta y Jugar = No los llamamos A y B; podemos tener en un antecedente tanto A, como B, o como AB, correspondiente al hecho de que se cumplan ambas en el antecedente.

In [4]:
# Create all the combinations
clases = []
for i in range(1,4):
#for i in range(1, len(list(datos.columns[1:]))+1):
# Reduce the number of examples to 3 iterations
    combis = list(combinations(datos.columns[1:], i))  
    for j, clase in enumerate(combis):        
        clases.append(list(clase[:i]))

Ahora, iteramos sobre cada una de estas identidades *(o clases en este script)* y creamos los item_sets determinando su cobertura. 

In [5]:
table = btable(max_width=3000)
table.column_headers = ['Item_Set', 'Cobertura']

item_sets = []
for i, cl in enumerate(clases):
    # Para cada combinacion de clases
    idx = list(datos[cl].drop_duplicates().index)
    atributos = datos[cl].loc[idx]
   
    for j in range(len(atributos)):
        
        atributo = list(atributos.iloc[j,:].values)       
        item_set = Item_Set(cl, atributo)
        item_set.calcular_cobertura(datos=datos)
        
        if item_set.cobertura >= COBERTURA_MINIMA:
            table.append_row([item_set.nombre, item_set.cobertura])
            item_sets.append(item_set)
            
print(table)

+----------------------------------------------------------------+-----------+
|                            Item_Set                            | Cobertura |
+----------------------------------------------------------------+-----------+
|                   ['Ambiente'] = ['soleado']                   |     5     |
+----------------------------------------------------------------+-----------+
|                   ['Ambiente'] = ['nublado']                   |     4     |
+----------------------------------------------------------------+-----------+
|                  ['Ambiente'] = ['lluvioso']                   |     5     |
+----------------------------------------------------------------+-----------+
|                   ['Temperatura'] = ['alta']                   |     4     |
+----------------------------------------------------------------+-----------+
|                  ['Temperatura'] = ['media']                   |     6     |
+---------------------------------------------------

Ahora creamos la **clase** ```Regla```, para generar **objetos** regla, que almacenaremos tanto en la **lista** ```reglamento[]``` que almacene todas las reglas como en el **atributo lista** ```Item_Set.reglas``` que almacena las reglas para cada item_set

In [6]:
class Regla:      

    def __init__(self, antecedentes, consecuentes):
        
        nombre = 'SI ' + str(antecedentes[0][0]) + ' = ' + str(antecedentes[0][1])
        for a in range(1, len(antecedentes)):
            nombre += ' Y ' + str(antecedentes[a][0]) + ' = ' + str(antecedentes[a][1]) 
        
        nombre += ' ENTONCES ' + str(consecuentes[0][0]) + ' = ' + str(consecuentes[0][1])
        for c in range(1, len(consecuentes)):
            nombre += ' Y ' + str(consecuentes[c][0]) + ' = ' + str(consecuentes[c][1])
        
        self.nombre = nombre
        self.antecedentes = antecedentes
        self.consecuentes = consecuentes
        self.confianza = 0
        self.soporte = 0
        
    def calcular_confianza(self, datos):
        '''
        Porcentaje de ejemplos que satisfacen el antecedente y consecuente 
        de la regla entre aquellos que solo satisfacen el antecedente
        '''
        numerador = 0
        denominador = 0
    
        antecedentes = self.antecedentes
        consecuentes = self.consecuentes
        
        for antecedente in self.antecedentes:
            
            # Buscamos en los datos filtrando por todos los antecedentes
            if 'si_a' not in locals():
                si_a = datos[datos[antecedente[0]] == antecedente[1]]    
            
            else: si_a = si_a[si_a[antecedente[0]] == antecedente[1]]
            
        denominador = len(si_a)
        
        for consecuente in self.consecuentes:
            
            # Buscamos en los datos filtrando por todos los consecuentes
            # Partimos de los datos que ya tienen filtrados los antecedentes 
            if 'si_c' not in locals():
                si_c = si_a[si_a[consecuente[0]] == consecuente[1]]
                
            else: si_c = si_c[si_c[consecuente[0]] == consecuente[1]]
            
        numerador = len(si_c)
            
        self.confianza = np.round((numerador / denominador), 2)
        self.soporte = np.round((numerador / EJEMPLOS), 2)
        

In [8]:
# Aprender Reglas
# ===============
reglamento = []

table = btable(max_width=3000)
table.column_headers = ['Regla', 'Cobertura']

# Iterar sobre los Item-sets que tengan al menos 2 items
for item_set in item_sets:
    
    if len(item_set.atributos) > 1:
    
        reglas = []
        #combinaciones = [list(par) for par in zip(item_set.clases, item_set.atributos)]
        combinaciones = [par for par in zip(item_set.clases, item_set.atributos)]
        
        # Generar todos los posibles antecedentes
        antecedentes = []
        for m in range(1, len(combinaciones)):   
            antecedentes += [list(a[:m]) for a in combinations(combinaciones, m)]
        
        
        # Caso de solo 2 unidades básicas A y B: solo existe la regla A -> B
        if len(antecedentes) < 3:
                
                antecedente = antecedentes[0]
                consecuente = antecedentes[1]
                nueva_regla = Regla(antecedente, consecuente)
                nueva_regla.calcular_confianza(datos)
                
                if nueva_regla.confianza > CONFIANZA_MINIMA:
                    reglas.append(nueva_regla)
                    item_set.reglas = reglas
                
                    table.append_row([nueva_regla.nombre, '%.3f' % nueva_regla.confianza])
                    reglamento.append(nueva_regla)
                
        else:
            
            # Para el resto de caos
            for antecedente in antecedentes:       
                    
                # Posibles consecuentes para el antecedente
                t = len(item_set.atributos)
                r = len(antecedente)
                candidatos = [a for a in antecedentes if len(a) == (t-r) and antecedente not in a]
                
                # Asociar consecuentes
                consecuentes = []
                for _, us in enumerate(candidatos):
                    
                    shouldbreak = False
                    for u in us:
                            
                        if shouldbreak: break
                        for a in antecedente:
                            
                            if (a not in us) and (u not in antecedente):
                                
                                consecuente = us
                                consecuentes.append(consecuente)
                                shouldbreak = True
                                break
                
                for consecuente in consecuentes:
                        
                        nueva_regla = Regla(antecedente, consecuente)
                        nueva_regla.calcular_confianza(datos)
                        
                        if nueva_regla.confianza > CONFIANZA_MINIMA:
                            reglas.append(nueva_regla)
                            item_set.reglas = reglas
                            
                            table.append_row([nueva_regla.nombre, '%.3f' % nueva_regla.confianza])
                            reglamento.append(nueva_regla)
                            
print(table)

+--------------------------------------------------------------+-----------+
|                            Regla                             | Cobertura |
+--------------------------------------------------------------+-----------+
|          SI Ambiente = nublado ENTONCES Jugar = si           |     1     |
+--------------------------------------------------------------+-----------+
|        SI Temperatura = alta ENTONCES Humedad = alta         |   0.75    |
+--------------------------------------------------------------+-----------+
|        SI Temperatura = media ENTONCES Humedad = alta        |   0.67    |
+--------------------------------------------------------------+-----------+
|       SI Temperatura = baja ENTONCES Humedad = normal        |     1     |
+--------------------------------------------------------------+-----------+
|        SI Temperatura = alta ENTONCES Viento = falso         |   0.75    |
+--------------------------------------------------------------+-----------+